In [ ]:
# from google.colab import drive
# drive.mount('/content/drive')
# %cd /content/drive/MyDrive/SMU_MITB_NLP/Group Project/NLP-Lyric-Generator/src/bin

In [ ]:
### Standard Imports
import numpy as np
import re
import sys
import os
from collections import Counter

import tensorflow as tf
from tensorflow.keras import layers

In [ ]:
### Custom Imports
sys.path.append('../')
import lib.utilities as utils
import lib.autoencoder_utilities as ae_utils

In [ ]:
### Text Parameters
start_token = '<cls>'
end_token = '<eos>'
pad_token = '<pad>'
unk_token = '<unk>'
newline_token = '<new>'
mask_token = '<mask>'

### General Parameters
random_seed = 2022
model_folder = '../../../autoencoder/lstm/v5'
model_name = 'ae_lstm_att_mask_song'

### Model Parameters
batch_size = 4
enc_dim, dec_dim = 256, 256
learn_rate = 0.001
epochs = 2
dropout = 0.05
recurrent_dropout = 0.05

In [ ]:
os.makedirs(model_folder, exist_ok=True)

In [ ]:
### Load Data
corpus = utils.load_corpus()
train_corpus, val_corpus, train_files, val_files = utils.split_corpus()

In [ ]:
songs = utils.split_text(corpus)
songs = [utils.tokenize_text(song, newline_token) for song in songs]
max_length = max(len(song) for song in songs)+2 # +2 for start token and end token
window_len = max_length

In [ ]:
### Pre-Processing Text
_, word_count, index_to_vocab, vocab_to_index, songs, songs_token_ind = utils.tokenize_corpus(corpus,
                                                                            window_length = window_len,
                                                                            end_token = end_token,
                                                                            start_token = start_token,
                                                                            pad_token = pad_token,
                                                                            unk_token = unk_token,
                                                                            newline_token = newline_token,
                                                                            mask_token = mask_token,
                                                                            padding = False)
vocab_size = len(word_count)

train_words, _, _, _, train_songs, train_songs_token_ind = utils.tokenize_corpus(train_corpus,
                                                                       window_length = window_len,
                                                                       index_to_vocab = index_to_vocab,
                                                                       vocab_to_index = vocab_to_index,
                                                                       end_token = end_token,
                                                                       start_token = start_token,
                                                                       pad_token = pad_token,
                                                                       unk_token = unk_token,
                                                                       newline_token = newline_token,
                                                                       mask_token = mask_token,
                                                                       padding = False)

val_words, _, _, _, val_songs, val_songs_token_ind = utils.tokenize_corpus(val_corpus,
                                                           window_length = window_len,
                                                           index_to_vocab = index_to_vocab,
                                                           vocab_to_index = vocab_to_index,
                                                           end_token = end_token,
                                                           start_token = start_token,
                                                           pad_token = pad_token,
                                                           unk_token = unk_token,
                                                           newline_token = newline_token,
                                                           mask_token = mask_token,
                                                           padding = False)

In [ ]:
### Need to reorg - Encoder has whole song, decoder has all prev tokens, y is the next token

train_x_encoder, train_x_decoder, train_y = ae_utils.construct_song_seq(train_songs_token_ind,
                                                                        max_len = max_length,
                                                                        pad_index = vocab_to_index[pad_token],
                                                                        n_copies = 10,
                                                                        mask_prob = 0.1,
                                                                        mask_index = vocab_to_index[mask_token],
                                                                        start_index = vocab_to_index[start_token],
                                                                        end_index = vocab_to_index[end_token],
                                                                        random_seed = random_seed)

val_x_encoder, val_x_decoder, val_y = ae_utils.construct_song_seq(val_songs_token_ind,
                                                                    max_len = max_length,
                                                                    pad_index = vocab_to_index[pad_token],
                                                                    n_copies = 10,
                                                                    mask_prob = 0.1,
                                                                    mask_index = vocab_to_index[mask_token],
                                                                    start_index = vocab_to_index[start_token],
                                                                    end_index = vocab_to_index[end_token],
                                                                    random_seed = random_seed)

In [ ]:
rand_int = np.random.randint(0, len(train_x_encoder), 1)[0]
print([index_to_vocab.get(x) for i, x in enumerate(train_x_encoder[rand_int]) if i < 30])
print(index_to_vocab.get(train_x_decoder[rand_int]))
print([index_to_vocab.get(train_y[rand_int])])

print('*'*10)

rand_int = np.random.randint(0, len(val_x_encoder), 1)[0]
print([index_to_vocab.get(x) for i, x in enumerate(val_x_encoder[rand_int]) if i < 30])
print(index_to_vocab.get(val_x_decoder[rand_int]))
print([index_to_vocab.get(val_y[rand_int])])

In [ ]:
train_dataset = ae_utils.construct_datasets(train_x_encoder, train_x_decoder, train_y,
                                            random_seed = random_seed,
                                            batch_size = batch_size,
                                            vocab_size = vocab_size)
val_dataset = ae_utils.construct_datasets(val_x_encoder, val_x_decoder, val_y,
                                            random_seed = random_seed,
                                            batch_size = batch_size,
                                            vocab_size = vocab_size)

In [ ]:
# Encoder
encoder_input = layers.Input(shape=(window_len,vocab_size), name = 'encoder_input')

# Return state in addition to output
encoder_output, encoder_hidden_state, encoder_cell_state = layers.LSTM(enc_dim,
                                                                       dropout = dropout, recurrent_dropout = recurrent_dropout,
                                                                       return_state=True, name = "encoder_lstm")(encoder_input)

# Decoder
decoder_input = layers.Input(shape=(vocab_size), name = 'decoder_input')

# Pass the encoder state to a new LSTM, as initial state
decoder_output = layers.LSTM(dec_dim,
                             dropout = dropout, recurrent_dropout = recurrent_dropout,
                             name="decoder_lstm")(decoder_input, initial_state=[encoder_hidden_state, encoder_cell_state])

# Attention
attention_context_vector = tf.keras.layers.Attention(name = 'attention')(inputs = [decoder_output, encoder_output])

# Output
output = layers.Dense(vocab_size, name = 'output', activation = 'softmax')(tf.concat([decoder_output, attention_context_vector], 1))

model = tf.keras.Model((encoder_input, decoder_input), output, name = model_name)
model.summary()

In [ ]:
model.compile(loss = 'categorical_crossentropy',
              optimizer = tf.keras.optimizers.Adam(learning_rate=learn_rate),
              metrics = ['accuracy'])

In [ ]:
### Callbacks
callback_es = tf.keras.callbacks.EarlyStopping(
    monitor='val_loss',
    min_delta=0,
    patience=10,
    verbose=1,
    mode='min',
    baseline=None,
    restore_best_weights=True
)

callback_mc = tf.keras.callbacks.ModelCheckpoint(
    filepath=model_folder+'/weights.{epoch:02d}-{val_loss:.2f}-{val_accuracy:.2f}.hdf5',
    save_weights_only=True,
    monitor='val_loss',
    mode='min',
    save_best_only=True)

In [ ]:
history = model.fit(x = train_dataset, validation_data = val_dataset, epochs = epochs, callbacks = [callback_es, callback_mc])

In [ ]:
model.save_weights(f'{model_folder}/final_weights.hdf5')

In [ ]:
#model.load_weights(f'{model_folder}/final_weights.hdf5')

In [ ]:
prompts = ['Whenever I think back', 'And so this I know',
           'I am tired of being what you want me to be', 'Feeling so faithless, lost under the surface',
           'Relight our fire, we will find our way', 'We will rise stronger together']
result_strings = {}
results = {}
for prompt in prompts:
    result_str, result = utils.generate_text(model,
                                             ae_utils.ind_to_input_fun, ae_utils.update_input_fun,
                                             start_string = prompt,
                                             window_length = window_len,
                                             vocab_to_index_dict = vocab_to_index, index_to_vocab_dict = index_to_vocab,
                                             vocab_size = vocab_size,
                                             num_generate = 100, temperature = 1.0,
                                             random_seed = random_seed,
                                             end_token = end_token, start_token = start_token,
                                             pad_token = pad_token, unk_token = unk_token,
                                             newline_token = newline_token,
                                             depth = vocab_size,
                                             to_mask = True,
                                             mask_index = vocab_to_index[mask_token])
    result_strings[prompt] = result_str
    results[prompt] = result

In [ ]:
print(result_strings)

In [ ]:
for k, v in result_strings.items():
    with open(model_folder+f'/human_{model_name}-{utils.remove_punct(k.lower())}.txt', 'w') as f:
        f.write(v)